In [1]:
from sentinel1_login import sh_client_id, sh_client_secret

# SETUP THE SENTINEUL-HUB ACCESS ID'S

from sentinelhub import SHConfig

config = SHConfig()

config.sh_client_id = sh_client_id
config.sh_client_secret = sh_client_secret

if not config.sh_client_id or not config.sh_client_secret:
    print("Warning! To use Process API, please provide the credentials (OAuth client ID and client secret).")

random change


In [124]:
# READ SHAPEFILE AND EXTRACT BOUNDING BOX FROM IT 

import fiona 

shp_file = '/home/ajay/Downloads/multi_shapefile/POLYGON.shp'

c = fiona.open(shp_file)

print(c.crs['init'])

BoundingBox = list(c.bounds)

BoundingBox

epsg:4326


[-6.299626934836539, 53.35505040168215, -6.218104867732819, 53.36994427003995]

In [133]:
# IMPORT THE NECCESSARY PYHTON PACKAGES

from datetime import datetime , timedelta
import os

import matplotlib.pyplot as plt
import numpy as np
import PIL.Image

from matplotlib import pyplot as plt

from sentinelhub import SentinelHubCatalog
catalog = SentinelHubCatalog(config=config)


from sentinelhub import (
    CRS,
    BBox,
    Geometry,
    DataCollection,
    DownloadRequest,
    MimeType,
    filter_times,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

%matplotlib inline 

In [134]:
# LINK: S1GRD orthorectified linear gamma0 VV, ascending orbit direction, GeoTIFF in EPSG:32648 (UTM zone 48N)
# EVALSCRIPT TO QUERY VV AND VH, SET TO DECIBELS BETWEEN -30 TO 0


evalscript1="""//VERSION=3
function setup() {
  return {
    input: ["VV", "VH"],
    output: [{ id:"vv_band", bands: 1,sampleType: SampleType.FLOAT32},
             { id:"vh_band", bands: 1,sampleType: SampleType.FLOAT32}] 
  }
}

function evaluatePixel(samples) {

  var vvdB = toDb(samples.VV)
  var vhdB = toDb(samples.VH)

  return {
     vv_band: [vvdB],
     vh_band: [vhdB]
  }
}

// displays VV in decibels from -30 to 0
function toDb(linear) {
  var log = 10 * Math.log(linear) / Math.LN10
  return Math.max(0, (log + 30) / 30)
}
"""

In [127]:
# LINK: S1GRD orthorectified linear gamma0 VV, ascending orbit direction, GeoTIFF in EPSG:32648 (UTM zone 48N)
# EVALSCRIPT TO QUERY VV AND VH, SET TO DECIBELS BETWEEN -20 TO 0


# evalscript1=
"""//VERSION=3
function setup() {
  return {
    input: ["VV", "VH"],
    output: [{ id:"vv_band", bands: 1,sampleType: SampleType.FLOAT32},
             { id:"vh_band", bands: 1,sampleType: SampleType.FLOAT32}] 
  }
}

function evaluatePixel(samples) {

  var vvdB = toDb(samples.VV)
  var vhdB = toDb(samples.VH)

  return {
     vv_band: [vvdB],
     vh_band: [vhdB]
  }
}

// displays VV in decibels from -20 to 0
function toDb(linear) {
  // the following commented out lines are simplified below
  // var log = 10 * Math.log(linear) / Math.LN10
  // var val = Math.max(0, (log + 20) / 20)
  return Math.max(0, Math.log(linear) * 0.21714724095 + 1)
}
"""

'//VERSION=3\nfunction setup() {\n  return {\n    input: ["VV", "VH"],\n    output: [{ id:"vv_band", bands: 1,sampleType: SampleType.FLOAT32},\n             { id:"vh_band", bands: 1,sampleType: SampleType.FLOAT32}] \n  }\n}\n\nfunction evaluatePixel(samples) {\n\n  var vvdB = toDb(samples.VV)\n  var vhdB = toDb(samples.VH)\n\n  return {\n     vv_band: [vvdB],\n     vh_band: [vhdB]\n  }\n}\n\n// displays VV in decibels from -20 to 0\nfunction toDb(linear) {\n  // the following commented out lines are simplified below\n  // var log = 10 * Math.log(linear) / Math.LN10\n  // var val = Math.max(0, (log + 20) / 20)\n  return Math.max(0, Math.log(linear) * 0.21714724095 + 1)\n}\n'

In [135]:
# ENTER TIME FOR INDIVIDUAL TIME SEARCH 
search_date  = datetime(2020, 10, 5)

# +/- 4 DAYS TO THE DATE TO CREATE A TIME RANGE FOR THE SEARCH
search_time_interval = (search_date - timedelta(days=4), search_date + timedelta(days=4))
print(search_time_interval)

(datetime.datetime(2020, 10, 1, 0, 0), datetime.datetime(2020, 10, 9, 0, 0))


In [ ]:
# SET TIME FOR LARGE TIME SARCHES
search_date1 = datetime(2023, 1, 1)
search_date2 = datetime(2023, 8 ,2)

search_time_interval = (search_date1, search_date2)

print(search_time_interval)

In [136]:
# FOR ASCENDING ORBIT DIRECTION
data_collection_asc = DataCollection.SENTINEL1_IW_ASC

# FOR DESCENDING ORBIT DIRECTION
data_collection_dsc = DataCollection.SENTINEL1_IW_DES


# SET THE BOUNDING BOX OR GEOMETRY
bbox = BBox(bbox=BoundingBox, crs=CRS.WGS84)

print(bbox)


-6.299626934836539,53.35505040168215,-6.218104867732819,53.36994427003995


In [ ]:
# # CODE TO VIEW THUMBNAIL OF THE QUERY TO CHECK QUALITY BEFORE PROCEEDING TO DOWNLOAD

# catalog = SentinelHubCatalog(config=config)

# # catalog.search()

# def get_s3_tile_paths(search_bbox, search_time_interval, data_collection, config):
#     results = catalog.search(collection=data_collection, bbox=search_bbox, time=search_time_interval)

#     return [result for result in results]
#     # return  [result["assets"]["thumbnail"] for result in results]

# preview_list = get_s3_tile_paths(bbox, search_time_interval, data_collection_asc, config)
# print(preview_list[0])
# print(len(preview_list))

In [137]:
# CODE TO VIEW THUMBNAIL OF THE QUERY TO CHECK QUALITY BEFORE PROCEEDING TO DOWNLOAD

search_iterator = catalog.search(collection=data_collection_asc, bbox=bbox, time=search_time_interval, fields={"include": ["id","properties.datetime", "assets.thumbnail", "assets.s3"], "exclude" : []})

results = list(search_iterator)

print("Total number of results:", len(results))

results

Total number of results: 4


[{'assets': {'s3': {'href': 's3://sentinel-s1-l1c/GRD/2020/10/4/IW/DV/S1A_IW_GRDH_1SDV_20201004T180645_20201004T180710_034650_040911_7F8C/',
    'title': 's3',
    'type': 'inode/directory'},
   'thumbnail': {'href': 'https://roda.sentinel-hub.com/sentinel-s1-l1c/GRD/2020/10/4/IW/DV/S1A_IW_GRDH_1SDV_20201004T180645_20201004T180710_034650_040911_7F8C/preview/quick-look.png',
    'title': 'thumbnail',
    'type': 'image/png'}},
  'id': 'S1A_IW_GRDH_1SDV_20201004T180645_20201004T180710_034650_040911_7F8C',
  'properties': {'datetime': '2020-10-04T18:06:45Z'}},
 {'assets': {'s3': {'href': 's3://sentinel-s1-l1c/GRD/2020/10/4/IW/DV/S1A_IW_GRDH_1SDV_20201004T180645_20201004T180710_034650_040911_3BBC/',
    'title': 's3',
    'type': 'inode/directory'},
   'thumbnail': {'href': 'https://roda.sentinel-hub.com/sentinel-s1-l1c/GRD/2020/10/4/IW/DV/S1A_IW_GRDH_1SDV_20201004T180645_20201004T180710_034650_040911_3BBC/preview/quick-look.png',
    'title': 'thumbnail',
    'type': 'image/png'}},
  'id'

In [ ]:
# AWS COMMADLINE CODE WITH THE S3 PATH OF TILE TO DOWNLOAD .SAFE FORMAT DATA 
# USE LS INSTEAD OF CP BELOW TO LIST THE FILES IN THE FOLDER 

# s3 cp  s3://sentinel-s1-l1c/GRD/2020/11/28/IW/DV/S1B_IW_GRDH_1SDV_20201128T170511_20201128T170536_024468_02E88E_E44D  ./  --request-payer --recursive

In [140]:
# CHECK HOW MANY UNIQUE ACQUISITIONS ARE AVAILABLE FOR THE GIVEN TIME PERIOD


time_difference = timedelta(hours=1)

all_timestamps = search_iterator.get_timestamps()
unique_acquisitions = filter_times(all_timestamps, time_difference)

unique_acquisitions

[datetime.datetime(2020, 10, 3, 18, 14, 16, tzinfo=tzutc()),
 datetime.datetime(2020, 10, 4, 18, 6, 45, tzinfo=tzutc())]

In [141]:
resolution = 10

size = bbox_to_dimensions(bbox, resolution=resolution)

In [143]:
import ast

# PATH TO SAVE THE DATA 
data_path = ""

# DATA COLLECTION TO QUERY (ASCENDING / DESCENDING)
# ONLY ASCENDING WORKS FOR NOW

data_collection = data_collection_asc

# DO YOU NEED ORTHO RECTIFICATION? TRUE/FALSE
orthorectification = "True"

# SELECT BACKSCATTER COEFFICIENT (DEFAULT GAMMA0_ELLIPSOID)
# OPTIONS (BETA0, SIGMA0_ELLIPSOID, GAMMA0ELLIPSOID, GAMMA0_TERRAIN)
back_coeff = "GAMMA0_ELLIPSOID"


# SELECT SPECKLE FILTERING (DEFAULT NONE)
# speckle_filter =  """{"type":"NONE"}"""

# UNCOMMENT THE FOLLOWING LINE TO ENABLE SPECKLE FILER AND CHANGE FILTER SIZE AS NEEDED
speckle_filter = """{"type":"LEE","windowSizeX":5,"windowSizeY":5}"""
speckle_filter = ast.literal_eval(speckle_filter)


speckle_filter_dict = {"speckleFilter" : speckle_filter}

print(speckle_filter_dict)

print(type(speckle_filter_dict))






{'speckleFilter': {'type': 'LEE', 'windowSizeX': 5, 'windowSizeY': 5}}
<class 'dict'>


In [145]:

# other_args = f'{{\"orthorectify\" :\"{orthorectification}\",\"backCoeff\": \"{back_coeff}\", \"speckleFilter\": {{\"type\": \"{speckle_filter}\"}}}}'

other_args = f'{{\"orthorectify\" :\"{orthorectification}\",\"backCoeff\": \"{back_coeff}\"}}'

print((other_args))
print(type(other_args))

other_args = ast.literal_eval(other_args)

print(other_args)
print(type(other_args))

other_args.update(speckle_filter_dict)

print(other_args)
print(type(other_args))

new_dict = {"processing" : other_args}

# print(new_dict["processing"]['speckleFilter']['type'])

{"orthorectify" :"True","backCoeff": "GAMMA0_ELLIPSOID"}
<class 'str'>
{'orthorectify': 'True', 'backCoeff': 'GAMMA0_ELLIPSOID'}
<class 'dict'>
{'orthorectify': 'True', 'backCoeff': 'GAMMA0_ELLIPSOID', 'speckleFilter': {'type': 'LEE', 'windowSizeX': 5, 'windowSizeY': 5}}
<class 'dict'>


In [146]:
process_requests = []

for timestamp in unique_acquisitions:
    request = SentinelHubRequest(
        data_folder = "/home/ajay/Documents/Sentinel_thumbnails/thumnbails",
        evalscript=evalscript1,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=data_collection_asc,
                time_interval=(timestamp - time_difference, timestamp + time_difference),
                # other_args={"processing": {"orthorectify": "True","backCoeff": "GAMMA0_ELLIPSOID","speckleFilter": {"type": "NONE"}}}
                other_args= new_dict
            )
        ],
        responses=[
        SentinelHubRequest.output_response('vv_band', MimeType.TIFF),
         SentinelHubRequest.output_response('vh_band', MimeType.TIFF),
        ],

        bbox=bbox,

        # geometry=final_aoidata,
        size=size,
        config=config,
    )
    process_requests.append(request)

print(len(process_requests))

2


In [147]:
# %%time

client = SentinelHubDownloadClient(config=config)

download_requests = [request.download_list[0] for request in process_requests]

data = client.download(download_requests)

# data[0].shape